In [1]:
import pandas as pd
import numpy as np
import sklearn.metrics as skm


In [3]:
errors = pd.read_csv('../errors.csv')
errors.index = ['Cosine','Jaccard','Pearson','PIP','MPIP']
errors.columns = ['MAE','RMSE','Precision','Recall','f1-score']
errors

,MAE,RMSE,Precision,Recall,f1-score
Cosine,1.370069,1.914075,0.335744,0.273264,0.245345
Jaccard,0.942397,1.205746,0.243128,0.302515,0.217494
Pearson,1.095055,5.084220,0.299624,0.328616,0.270832
PIP,0.887194,1.174997,0.365533,0.331017,0.266181
MPIP,0.849342,1.137702,0.325495,0.343917,0.271720


In [2]:
cosine = pd.read_csv('../cosine.csv').to_numpy()
pearson = pd.read_csv('../pearson.csv').to_numpy()
jaccard = pd.read_csv('../jaccard.csv').to_numpy()
pip = pd.read_csv('../pip.csv').to_numpy()
mpip =pd.read_csv('../mpip.csv',delimiter=',').to_numpy()
train_data = pd.read_csv('../train.data',delimiter=',')
test_data = pd.read_csv('../test.data',delimiter=',').to_numpy()
matrix = np.zeros((943,1682),dtype=int)
users = test_data.transpose()[0]
items = test_data.transpose()[1]
ratings = test_data.transpose()[2]
for row in train_data.to_numpy():
    matrix[row[0]-1,row[1]-1] = row[2]

In [6]:
def PredRating(user,item,similarity):
    try:
        top = similarity[user].argsort()[1:100]
    except IndexError:
        sum,count = 0,0
        for j in range(1682):
            if matrix[user,j] != 0:
                count+=1
                sum+=1
        return sum/count
    temp,avgUh,simi = [],[],[]
    for i in top:
        if matrix[i,item] !=0:
            temp.append(i)
            simi.append(similarity[user,i])
    temp.append(user)
    for i in temp:
        sum,count = 0,0
        for j in range(1682):
            if matrix[i,j] != 0:
                count+=1
                sum+=matrix[i,j]
        avgUh.append(sum/count)
    avgU = np.nan_to_num(np.array(avgUh.pop()))
    temp.pop()
    if len(temp)==0:
        return avgU
    simi = np.nan_to_num(np.array(simi))
    num = (simi*(avgUh-avgU)).sum()
    den = simi.sum()
    # for i in range(len(temp)):
    #     num+=similarity[user,temp[i]]*avgUh[i]
    #     den+=similarity[user,temp[i]]
    try:
        result = (round((avgU+num/den),0))
    except ZeroDivisionError:
        result = 0
    return result


In [7]:
cosinePred = []
jaccardPred,pearsonPred,pipPred,mpipPred = [],[],[],[]
user = []
item = []
rating = []
for i in range(len(users)):
    temp = PredRating(users[i]-1,items[i]-1,cosine)
    jaccardPred.append(PredRating(users[i]-1,items[i]-1,jaccard))
    pearsonPred.append(PredRating(users[i]-1,items[i]-1,pearson))
    pipPred.append(PredRating(users[i]-1,items[i]-1,pip))
    mpipPred.append(PredRating(users[i]-1,items[i]-1,mpip))
    cosinePred.append(temp)
    user.append(users[i])
    item.append(items[i])
    rating.append(ratings[i])

<ipython-input-6-b320d13bfef8>:35: RuntimeWarning: invalid value encountered in double_scalars
  result = (round((avgU+num/den),0))


In [8]:
result = pd.DataFrame({'user':user,
                        'item':item,
                        'rating':rating,
                        'cosine':cosinePred,
                        'jaccard':jaccardPred,
                        'pearson':pearsonPred,
                        'pip':pipPred,
                        'mpip':mpipPred
                        }).fillna(0)

In [16]:
result['cosine'] = round(result['cosine'],0)
result['jaccard'] = round(result['jaccard'],0)
result['pearson'] = round(result['pearson'],0)
result['pip'] = round(result['pip'],0)
result['mpip'] = round(result['mpip'],0)
result.to_csv('../results.csv',sep=',',index=False)

In [18]:
resTrans=result.to_numpy().transpose()
f1_score,mae,recall,precision,rmse = [],[],[],[],[]


In [19]:
for i in range(3,8):
    f1_score.append(skm.f1_score(resTrans[2],resTrans[i],average='weighted'))
    mae.append(skm.mean_absolute_error(resTrans[2],resTrans[i]))
    rmse.append(skm.mean_squared_error(resTrans[2],resTrans[i],squared=False))
    recall.append(skm.recall_score(resTrans[2],resTrans[i],average='weighted'))
    precision.append(skm.precision_score(resTrans[2],resTrans[i],average='weighted'))

C:\Users\gurvi\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\gurvi\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\gurvi\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [20]:
errors = pd.DataFrame({'MAE':mae,'RMSE':rmse,'Precision':precision,'Recall':recall,'f1-score':f1_score})


In [23]:
errors.index = ['Cosine','Jaccard','Pearson','PIP','MPIP']
errors

,mae,rmse,precision,recall,f1-score
cosine,1.370069,1.914075,0.335744,0.273264,0.245345
jaccard,0.942397,1.205746,0.243128,0.302515,0.217494
pearson,1.095055,5.084220,0.299624,0.328616,0.270832
pip,0.887194,1.174997,0.365533,0.331017,0.266181
mpip,0.849342,1.137702,0.325495,0.343917,0.271720


In [22]:
errors.to_csv('../errors.csv',sep=',',index=False)

In [46]:
def recommendtion(user,similarity,matrix):
    items = pd.read_csv('../u.item',delimiter='|',names=[str(i) for i in range(1,25)]).to_numpy()
    items = items[:,1]
    topuser = similarity[user-1].argsort()[:10]
    #print(topuser)
    movies = []
    for i in topuser:
        topmovUI = matrix[i].argsort()[:10]
        #print(topmovUI)
        for j in topmovUI:
            if matrix[user-1,j] == 0  and (matrix[i,j] != 0 or matrix[i,j] != 1):
                #print(j)
                #print(items[j])
                movies.append(items[j])
    return movies

In [47]:
cosine = pd.read_csv('../cosine.csv',delimiter=',').to_numpy()
res = recommendtion(1,cosine,matrix)

In [48]:
res

['Chungking Express (1994)',
 'Heidi Fleiss: Hollywood Madam (1995) ',
 'Truman Show, The (1998)',
 'Old Man and the Sea, The (1958)',
 'Innocents, The (1961)',
 'Farewell to Arms, A (1932)',
 'Last Time I Saw Paris, The (1954)',
 'They Made Me a Criminal (1939)',
 'Umbrellas of Cherbourg, The (Parapluies de Cherbourg, Les) (1964)',
 'Truman Show, The (1998)',
 'Old Man and the Sea, The (1958)',
 'Innocents, The (1961)',
 'Farewell to Arms, A (1932)',
 'Last Time I Saw Paris, The (1954)',
 'They Made Me a Criminal (1939)',
 'Umbrellas of Cherbourg, The (Parapluies de Cherbourg, Les) (1964)',
 "I'm Not Rappaport (1996)",
 'Some Kind of Wonderful (1987)',
 'Chungking Express (1994)',
 'Heidi Fleiss: Hollywood Madam (1995) ',
 'Truman Show, The (1998)',
 'Old Man and the Sea, The (1958)',
 'Innocents, The (1961)',
 'Farewell to Arms, A (1932)',
 'Last Time I Saw Paris, The (1954)',
 'They Made Me a Criminal (1939)',
 'Umbrellas of Cherbourg, The (Parapluies de Cherbourg, Les) (1964)',
 'H

In [6]:
items = pd.read_csv('../u.item',delimiter='|',names=[str(i) for i in range(29)],encoding='latin-1')['1'].to_numpy()
filterwords = pd.read_csv('../offensive word.txt').to_numpy()
def top(user,similarity,matrix,items):
    topMovies = []
    simUsers = similarity[user].argsort()[:5]+1
    topMovies = matrix[simUsers[:]].argsort()[:10]
    titles = []
    years = []
    for i in topMovies:
        for j in i:
            itemsTemp = (str(items[j])).split('(')
            titles.append(itemsTemp[0])
            years.append(itemsTemp[1].rstrip(')'))
            if(len(titles)>20):
              break
        if(len(titles)>20):
              break
    years = np.array(years).argsort()
    titles = np.array(titles)
    titles = titles[years[:]]
    for i in range(len(titles)):
      temp = str(titles[i]).split()
      if temp in filterwords:
        titles.pop(index=i)
    return titles

In [7]:
print(top(660,cosine,matrix,items))
print(top(660,jaccard,matrix,items))
print(top(660,pearson,matrix,items))


['Farewell to Arms, A ' 'They Made Me a Criminal ' 'Mark of Zorro, The '
 'Last Time I Saw Paris, The ' 'Old Man and the Sea, The '
 'Innocents, The ' 'Up in Smoke ' 'Some Kind of Wonderful '
 'Double Happiness ' 'Cobb ' 'Mrs. Parker and the Vicious Circle '
 'Death and the Maiden ' 'Chungking Express ' 'Tank Girl ' 'Toy Story '
 'Heidi Fleiss: Hollywood Madam ' "I'm Not Rappaport " 'Twelfth Night '
 'Faithful ' 'Truman Show, The ' 'Umbrellas of Cherbourg, The ']
['Farewell to Arms, A ' 'They Made Me a Criminal ' 'Mark of Zorro, The '
 'Last Time I Saw Paris, The ' 'Innocents, The ' 'Some Kind of Wonderful '
 "Jupiter's Wife " 'Chungking Express ' 'Cobb ' 'Double Happiness '
 'Mrs. Parker and the Vicious Circle ' 'Tank Girl ' 'Safe '
 'Heidi Fleiss: Hollywood Madam ' 'Faithful ' 'Glimmer Man, The '
 'Surviving Picasso ' "I'm Not Rappaport " 'Twelfth Night '
 'Truman Show, The ' 'Umbrellas of Cherbourg, The ']
['Farewell to Arms, A ' 'They Made Me a Criminal ' 'Mark of Zorro, The '
 'La

In [8]:
print(top(660,pip,matrix,items))
print(top(660,mpip,matrix,items))

['Farewell to Arms, A ' 'They Made Me a Criminal ' 'Mark of Zorro, The '
 'Last Time I Saw Paris, The ' 'Old Man and the Sea, The '
 'Innocents, The ' 'Escape to Witch Mountain ' 'Up in Smoke '
 "Jupiter's Wife " 'Chungking Express ' 'Safe '
 'Heidi Fleiss: Hollywood Madam ' 'Surviving Picasso '
 "I'm Not Rappaport " 'Glimmer Man, The ' 'Twelfth Night '
 'Feeling Minnesota ' 'Get on the Bus ' 'Faithful ' 'Truman Show, The '
 'Umbrellas of Cherbourg, The ']
['Farewell to Arms, A ' 'They Made Me a Criminal ' 'Mark of Zorro, The '
 'Last Time I Saw Paris, The ' 'Old Man and the Sea, The '
 'Innocents, The ' 'Some Kind of Wonderful ' 'Chungking Express ' 'Cobb '
 'Mrs. Parker and the Vicious Circle ' 'Double Happiness '
 "Jupiter's Wife " 'Tank Girl ' 'Heidi Fleiss: Hollywood Madam '
 'Faithful ' 'Glimmer Man, The ' 'Surviving Picasso ' 'Twelfth Night '
 "I'm Not Rappaport " 'Truman Show, The ' 'Umbrellas of Cherbourg, The ']
